In [1]:
import pandas as pd

In [3]:
# 메인 데이터
app = pd.read_csv("./score_data/application_train_EXT_SOURCE.csv")

# 서브 테이블들
bureau = pd.read_csv("./score_data/bureau_final.csv")
credit = pd.read_csv("./score_data/credit_card_kpi_final.csv")
installment = pd.read_csv("./score_data/installment_kpi_final.csv")
pos = pd.read_csv("./score_data/pos_final.csv")
prev = pd.read_csv("./score_data/previous_final.csv")
bureau_active_agg = pd.read_csv("./score_data/df2_active_agg.csv")

In [ ]:
bureau_active_agg

In [ ]:
prev

In [ ]:
pos

In [ ]:
installment

In [ ]:
credit

In [4]:
# 각 테이블의 고객 ID 집합 생성
id_app = set(app['SK_ID_CURR'])
id_bureau = set(bureau['SK_ID_CURR'])
id_credit = set(credit['SK_ID_CURR'])
id_install = set(installment['SK_ID_CURR'])
id_pos = set(pos['SK_ID_CURR'])
id_prev = set(prev['SK_ID_CURR'])
id_bureau_active = set(bureau_active_agg['SK_ID_CURR'])

# 전체 교집합 ID 추출
common_ids = id_app & id_bureau & id_credit & id_install & id_pos & id_prev & id_bureau_active

# 결과 확인
print(f"모든 테이블에 공통으로 존재하는 고객 수: {len(common_ids)}")

모든 테이블에 공통으로 존재하는 고객 수: 72793


In [5]:
df = pd.read_csv('./score_data/merged_kpi_dataset_v2.csv')

In [6]:
# 공통 ID로 필터링
df_common = df[df['SK_ID_CURR'].isin(common_ids)].copy()

print("필터링된 데이터 shape:", df_common.shape)

필터링된 데이터 shape: (72793, 39)


In [7]:
df_common

,SK_ID_CURR,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,TARGET,CC_90PLUS,AVG_SK_DPD_x,AVG_CARD_USAGE_MONTHS,AVG_UTIL_RATIO,AVG_MIN_PAY_RATIO,...,DELTA_CREDIT_SUM_IF_INCREASED,DELTA_CREDIT_PCT_IF_INCREASED,OVERLAP,OVERLAP_YEARS,HAS_HIGH_RISK_DEBT,END_STATUS_SCORE,bureau_IS_ACTIVE_ESTIMATED,bureau_IS_CLOSED_ACTUAL,bureau_DAYS_CREDIT_ENDDATE,NO_CREDIT_HISTORY_FLAG
8,100011,0.587334,0.205747,0.751724,0,0.0,0.000000,74.0,0.302678,-0.200854,...,91242.000,1.689667,2.0,1.317808,0.0,-1.0,0.0,1.000000,-1293.0,0
19,100023,-1.000000,0.586617,0.477649,0,0.0,0.000000,8.0,0.000000,0.000000,...,562308.345,1.253185,11.0,16.320548,0.0,-1.0,3.0,0.769231,-365.0,0
31,100036,0.274422,0.627300,-1.000000,0,0.0,0.000000,12.0,0.000000,0.000000,...,0.000,NaN,0.0,0.000000,0.0,0.0,0.0,0.000000,-462.0,0
36,100043,0.842763,0.681699,0.754406,0,0.0,0.000000,33.0,0.487889,3.620759,...,0.000,NaN,0.0,-3.517808,0.0,0.0,0.0,1.000000,-1434.0,0
40,100047,-1.000000,0.306841,0.320163,1,0.0,0.000000,36.0,0.000000,0.000000,...,5437800.000,2.388327,4.0,21.471233,0.0,-2.0,3.0,0.400000,1301.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307497,456241,0.251096,0.651306,0.524496,0,0.0,0.000000,33.0,0.000000,0.000000,...,90000.000,0.666667,2.0,0.890411,0.0,-1.0,0.0,0.600000,-1623.0,0
307498,456242,-1.000000,0.746259,0.406617,0,0.0,0.000000,8.0,0.691240,0.041030,...,0.000,NaN,0.0,0.000000,0.0,0.0,1.0,0.000000,509.0,0
307500,456244,0.896042,0.789389,0.337673,0,0.0,0.000000,41.0,0.444874,0.018524,...,2590920.000,1.198548,20.0,26.621918,0.0,-6.0,2.0,0.826087,-1132.0,0
307502,456246,-1.000000,0.313334,0.735221,0,0.0,0.000000,8.0,0.097309,0.472287,...,135000.000,1.500000,2.0,1.057534,0.0,-2.0,1.0,0.666667,-235.0,0


In [ ]:
df

In [8]:
missing_count = df_common.isnull().sum()
missing_ratio = (df_common.isnull().mean() * 100).round(2)

missing_df = pd.DataFrame({
    'Missing_Count': missing_count,
    'Missing_Ratio(%)': missing_ratio
})

# 결측치 있는 컬럼만 보기
missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values(by='Missing_Ratio(%)', ascending=False)

print(missing_df)

                               Missing_Count  Missing_Ratio(%)
DELTA_CREDIT_PCT_IF_INCREASED          27287             37.49
bureau_DAYS_CREDIT_ENDDATE               591              0.81


In [ ]:
# 3개 모두 결측치인 행 필터링
cond = (
    df['CREDIT_CARD_HISTORY_YEARS'].isnull() &
    df['LOAN_HISTORY_YEARS'].isnull() &
    df['LOAN_HISTORY_YEARS(prev)'].isnull()
)

# 해당 조건을 만족하는 행만 추출
missing_ids_df = df.loc[cond, ['SK_ID_CURR']].drop_duplicates()

# 결과 출력
print(f"세 변수 모두 NULL인 고객 수: {missing_ids_df.shape[0]}")
print("고객 ID 예시:")
print(missing_ids_df.head())

세 변수 모두 NULL인 고객 수: 15407
고객 ID 예시:
    SK_ID_CURR
20      100024
26      100031
50      100058
64      100076
81      100096


In [12]:
df_common['SK_ID_CURR'].to_csv('SK_ID_70K.csv', index=False, encoding='utf-8-sig')

In [16]:
df['IS_OVERDUE_LONG'].info()
df['IS_OVERDUE_SHORT'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 307511 entries, 0 to 307510
Series name: IS_OVERDUE_LONG
Non-Null Count   Dtype  
--------------   -----  
263491 non-null  float64
dtypes: float64(1)
memory usage: 2.3 MB
<class 'pandas.core.series.Series'>
RangeIndex: 307511 entries, 0 to 307510
Series name: IS_OVERDUE_SHORT
Non-Null Count   Dtype  
--------------   -----  
263491 non-null  float64
dtypes: float64(1)
memory usage: 2.3 MB


In [ ]:
df_common['DELTA_CREDIT_PCT_IF_INCREASED']

8         1.689667
19        1.253185
31             NaN
36             NaN
40        2.388327
            ...   
307497    0.666667
307498         NaN
307500    1.198548
307502    1.500000
307503    8.998344
Name: DELTA_CREDIT_PCT_IF_INCREASED, Length: 72793, dtype: float64

In [ ]:
df_common['bureau_DAYS_CREDIT_ENDDATE']

8        -1293.0
19        -365.0
31        -462.0
36       -1434.0
40        1301.0
           ...  
307497   -1623.0
307498     509.0
307500   -1132.0
307502    -235.0
307503    1450.0
Name: bureau_DAYS_CREDIT_ENDDATE, Length: 72793, dtype: float64

In [19]:
df['DELTA_CREDIT_PCT_IF_INCREASED']

0          4.103036
1         10.148963
2               NaN
3               NaN
4               NaN
            ...    
307506          NaN
307507          NaN
307508     5.250000
307509          NaN
307510    12.407636
Name: DELTA_CREDIT_PCT_IF_INCREASED, Length: 307511, dtype: float64